# Why ensemble learning works?
#### Huang Chen

This notebook extends the notebook posted by David Solans in his github: [David Solans Ensemble Learning Github](https://github.com/dsolanno/UBDataScience-Machine-Learning/tree/master/Ensemble%20learning)

Ensemble learning helps improve machine learning results by combining several models. This approach allows the production of better predictive performance compared to a single model. That is why ensemble methods placed first in many prestigious machine learning competitions, such as the Netflix Competition, KDD 2009, and Kaggle.

Ensemble methods are meta-algorithms that combine several machine learning techniques into one predictive model in order to decrease variance (bagging), bias (boosting), or improve predictions (stacking).

Ensemble methods can be divided into two groups:

- *Sequential* ensemble methods where the base learners are generated sequentially (e.g. AdaBoost).
The basic motivation of sequential methods is to exploit the dependence between the base learners. The overall performance can be boosted by weighing previously mislabeled examples with higher weight.
- *Parallel* ensemble methods where the base learners are generated in parallel (e.g. Random Forest). 

The basic motivation of parallel methods is to exploit independence between the base learners since the error can be reduced dramatically by averaging.
Most ensemble methods use a single base learning algorithm to produce homogeneous base learners, i.e. learners of the same type, leading to homogeneous ensembles.

There are also some methods that use heterogeneous learners, i.e. learners of different types, leading to heterogeneous ensembles. In order for ensemble methods to be more accurate than any of its individual members, the base learners have to be as accurate as possible and as diverse as possible.

## Why ensemble learning?
To understand that why the generalization ability of an ensemble is usually much stronger than that of a single learner, there are three main reasons:
- The first reason is that, the training data might not provide sufficient information for choosing a single best learner. For example, there may be many learners perform equally well on the training data set. Thus, combining these learners may be a better choice. 
- The second reason is that, the search processes of the learning algorithms might be imperfect.For example, even if there exists a unique best hypothesis, it might be difficult to achieve since running the algorithms result in sub-optimal hypotheses. Thus, ensembles can compensate for such imperfect search processes. 
- The third reason is that, the hypothesis space being searched might not contain the true target function, while ensembles can give some good approximation. For example, it is well-known that the classification boundaries of decision trees are linear segments parallel to coordinate axes. If the target classification boundary is a smooth diagonal line, using a single decision tree cannot lead to a good result yet a good approximation can be achieved by combining a set of decision trees.

# Bagging
Bootstrap aggregating, often abbreviated as bagging, is a involves having each model in the ensemble vote with equal weight. In order to promote model variance, bagging trains each model in the ensemble using a randomly drawn subset of the training set. As an example, the random forest algorithm combines random decision trees with bagging to achieve very high classification accuracy.

One way to reduce the variance of an estimate is to average together multiple estimates. For example, we can train M different trees on different subsets of the data (chosen randomly with replacement) and compute the ensemble:
$$ f(x) = \frac{1}{M} \sum_{m=1}^{M}f_m(x)$$

Bagging uses bootstrap sampling to obtain the data subsets for training the base learners. For aggregating the outputs of base learners, bagging uses voting for classification and averaging for regression.

We can study bagging in the context of classification on the Iris dataset. We can choose two base estimators: a decision tree and a k-NN classifier. Figure 1 shows the learned decision boundary of the base estimators as well as their bagging ensembles applied to the Iris dataset.

- Accuracy: 0.63 (+/- 0.02) Decision Tree
- Accuracy: 0.70 (+/- 0.02) K-NN
- Accuracy: 0.64 (+/- 0.01) Bagging Tree
- Accuracy: 0.59 (+/- 0.07) Bagging K-NN





![alt text](bagging.png )
<p style="text-align: center;">Figure 1. Learned decision boundary of the base estimators and their bagging ensembles .</p>

The decision tree shows the axes’ parallel boundaries, while the $k=1$ nearest neighbors fit closely to the data points. The bagging ensembles were trained using 10 base estimators with 0.8 subsampling of training data and 0.8 subsampling of features.

The decision tree bagging ensemble achieved higher accuracy in comparison to the k-NN bagging ensemble. K-NN are less sensitive to perturbation on training samples and therefore they are called stable learners.

The figure also shows how the test accuracy improves with the size of the ensemble. Based on cross-validation results, we can see the accuracy increases until approximately 10 base estimators and then plateaus afterwards. Thus, adding base estimators beyond 10 only increases computational complexity without accuracy gains for the Iris dataset.

We can also see the learning curves for the bagging tree ensemble. Notice an average error of 0.3 on the training data and a U-shaped error curve for the testing data. The smallest gap between training and test errors occurs at around 80% of the training set size.

In random forests, each tree in the ensemble is built from a sample drawn with replacement (i.e. a bootstrap sample) from the training set. In addition, instead of using all the features, a random subset of features is selected, further randomizing the tree.

As a result, the bias of the forest increases slightly, but due to the averaging of less correlated trees, its variance decreases, resulting in an overall better model.


![alt text](random_forest.png )
<p style="text-align: center;">Figure 2. Example of classification of one sample by the trees on a random forest with n trees.</p>

In an extremely randomized trees algorithm randomness goes one step further: the splitting thresholds are randomized. Instead of looking for the most discriminative threshold, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows reduction of the variance of the model a bit more, at the expense of a slightly greater increase in bias.

# Python code

Here I add the code to support the above example explained by using Iris dataset. 

The following code shows the accuracy of using a single estimator of Decision Tree and KNN, it also prints the accuracy of using Bagging classfier based on 10 estimators of Decision Tree and KNN with 0.8 subsampling of training data and 0.8 subsampling of features. Such length of estimators(10), subsamples size(0.8) and subsamples featrues(0.8) are followed by the previously David's work. Also, by default in the Sklearn tutorial page the lenght of estimator is 10 as well. More details, it can be found by [Sscikit-learn Bagging Classifier tutorial page](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

In [31]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn import tree
from sklearn.model_selection import cross_val_score

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

#Single estimator Decision Tree 
clf_tree = tree.DecisionTreeClassifier(random_state=0)
clf_tree.fit(X, y)
scores_tree = cross_val_score(clf_tree, X, y)
print("The accuracy of using Desicion Tree is:",scores_tree.mean())

#Single estimator KNeighbors(KNN)
clf_KNN= KNeighborsClassifier()
clf_KNN.fit(X, y)
scores_KNN = cross_val_score(clf_KNN, X, y)
print("The accuracy of using KNN is:",scores_KNN.mean())

#By default 10 estimators of Decision Tree
clf_bagging_tree = BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(random_state=0),max_samples=0.8,max_features =0.8)
clf_bagging_tree.fit(X,y)
scores_bagging_tree = cross_val_score(clf_bagging_tree, X, y)
print("The accuracy of using Bagging technique based on Desicion Tree is:",scores_bagging_tree.mean())

#By default 10 estimators of KNN
clf_bagging_KNN = BaggingClassifier(base_estimator=KNeighborsClassifier(),max_samples=0.8,max_features =0.8)
clf_bagging_KNN.fit(X,y)
scores_bagging_KNN = cross_val_score(clf_bagging_KNN, X, y)
print("The accuracy of using Bagging technique based on KNN is:",scores_bagging_KNN.mean())

The accuracy of using Desicion Tree is: 0.698937908497
The accuracy of using KNN is: 0.753676470588
The accuracy of using Bagging technique based on Desicion Tree is: 0.692810457516
The accuracy of using Bagging technique based on KNN is: 0.706290849673


Now, I would like to try with an higher number of estimators, subsamples size and features. The default number of estimators is 10, the below code shows the accuracy of using Bagging based on 15 estimators of Decision Tree and KNN, also it uses the default number of sumbsamples size and features which are both $1.0$.

In [32]:
#Classfier base on 15 number
clf_bagging_tree = BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(random_state=0),n_estimators=15, random_state=0)
clf_bagging_tree.fit(X,y)
scores_bagging_tree = cross_val_score(clf_bagging_tree, X, y)
print("The accuracy of using Bagging technique based on Desicion Tree is:",scores_bagging_tree.mean())

clf_bagging_KNN = BaggingClassifier(KNeighborsClassifier(),n_estimators=15, random_state=0)
clf_bagging_KNN.fit(X,y)
scores_bagging_KNN = cross_val_score(clf_bagging_KNN, X, y)
print("The accuracy of using Bagging technique based on KNN is:",scores_bagging_KNN.mean())

The accuracy of using Bagging technique based on Desicion Tree is: 0.733660130719
The accuracy of using Bagging technique based on KNN is: 0.767156862745


The accurracy of Bagging+Decision Tree has incresed approximaly 0.04, also the another ona has incresed 0.061 which are much better than before. The detail explanations of why using multiple estimator is better are shown on the above David's work.
Conclusion, I just changed a little bit the parameters of BaggingClassifier, but the result has significantly improved as the above code result shows. 